In [1]:
using StatisticalRethinking, Distributed, JLD
using Mamba

# Data
line = Dict{Symbol, Any}()

howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [2]:
df2 = filter(row -> row[:age] >= 18, df);
line[:x] = convert(Array{Float64,1}, df2[:weight]);
line[:y] = convert(Array{Float64,1}, df2[:height]);
line[:xmat] = convert(Array{Float64,2}, [ones(length(line[:x])) line[:x]])

# Model Specification
model = Model(
  y = Stochastic(1,
    (xmat, beta, s2) -> MvNormal(xmat * beta, sqrt(s2)),
    false
  ),
  beta = Stochastic(1, () -> MvNormal([178, 0], [sqrt(10000), sqrt(100)])),
  s2 = Stochastic(() -> Uniform(0, 50))
)

# Initial Values
inits = [
  Dict{Symbol, Any}(
    :y => line[:y],
    :beta => [rand(Normal(178, 100)), rand(Normal(0, 10))],
    :s2 => rand(Uniform(0, 50))
  )
  for i in 1:3
]

# Tuning Parameters
scale1 = [0.5, 0.25]
summary1 = identity
eps1 = 0.5

scale2 = 0.5
summary2 = x -> [mean(x); sqrt(var(x))]
eps2 = 0.1

# Define sampling scheme

scheme = [
  Mamba.NUTS([:beta]),
  Mamba.Slice([:s2], 10)
]

setsamplers!(model, scheme)

# MCMC Simulation

sim = mcmc(model, line, inits, 10000, burnin=1000, chains=3)

MCMC Simulation of 10000 Iterations x 3 Chains...

Chain 1:   0% [0:12:07 of 0:12:07 remaining]
Chain 1:  10% [0:00:47 of 0:00:52 remaining]
Chain 1:  20% [0:00:49 of 0:01:02 remaining]
Chain 1:  30% [0:00:46 of 0:01:06 remaining]
Chain 1:  40% [0:00:41 of 0:01:08 remaining]
Chain 1:  50% [0:00:35 of 0:01:10 remaining]
Chain 1:  60% [0:00:29 of 0:01:12 remaining]
Chain 1:  70% [0:00:21 of 0:01:11 remaining]
Chain 1:  80% [0:00:14 of 0:01:12 remaining]
Chain 1:  90% [0:00:07 of 0:01:13 remaining]
Chain 1: 100% [0:00:00 of 0:01:13 remaining]

Chain 2:   0% [0:00:03 of 0:00:03 remaining]
Chain 2:  10% [0:00:50 of 0:00:56 remaining]
Chain 2:  20% [0:00:53 of 0:01:06 remaining]
Chain 2:  30% [0:00:49 of 0:01:10 remaining]
Chain 2:  40% [0:00:44 of 0:01:14 remaining]
Chain 2:  50% [0:00:38 of 0:01:16 remaining]
Chain 2:  60% [0:00:31 of 0:01:16 remaining]
Chain 2:  70% [0:00:23 of 0:01:17 remaining]
Chain 2:  80% [0:00:15 of 0:01:17 remaining]
Chain 2:  90% [0:00:08 of 0:01:17 remaining]
Cha

Object of type "Mamba.ModelChains"

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

[26.2849 115.148 0.87924; 29.4123 115.147 0.881363; … ; 27.246 110.527 0.978253; 24.484 110.529 0.982095]

[26.9095 110.19 0.989041; 27.1874 110.188 0.985401; … ; 25.1208 114.624 0.879535; 25.4718 114.626 0.883767]

[26.9325 114.764 0.886367; 27.4431 114.752 0.886708; … ; 23.8403 112.947 0.925217; 24.1728 113.247 0.918727]

Show draws summary

In [3]:
describe(sim)

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

Empirical Posterior Estimates:
            Mean         SD        Naive SE       MCSE        ESS   
     s2  26.1714703 1.986569905 0.01208987943 0.0198626212 9000.0000
beta[1] 113.8286394 1.904442259 0.01159006650 0.0357291909 2841.1179
beta[2]   0.9060929 0.041909992 0.00025505609 0.0007804733 2883.4942

Quantiles:
            2.5%         25.0%       50.0%        75.0%        97.5%   
     s2  22.58358300  24.78445274  26.0682816  27.44376123  30.30911426
beta[1] 110.13329090 112.53649822 113.8453639 115.11493739 117.55914961
beta[2]   0.82445606   0.87770506   0.9057476   0.93463918   0.98768827



End of `clip_38.0m.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*